In [3]:
import os
import dill as pickle
import json
import re
import string
import sys
from nltk.stem.porter import PorterStemmer
import logging

# Unpickle the Model

In [4]:
clf_path = '/Users/scottmcallister/Desktop/GitHub/fbo-scraper/utils/binaries/best_clf_accuracy.pkl'
with open(clf_path, 'rb') as f:
    pickled_model = pickle.load(f)

# Dummy Text

In [6]:
text = ['''
Request for Information
Software Composition Analysis Tool 

This is a Request for Information (RFI). This Sources Sought Notice is for informational and planning purposes only and shall not be construed as a solicitation or as an obligation or commitment by the Government. This announcement is not a request for proposals, and the Government is not committed to issue a solicitation or award a contract pursuant to this announcement or based on responses to this announcement. This notice is strictly for market research to assist the Government for planning purposes. The Government will not pay any costs incurred in the preparation of information for responding to this market survey or the Government’s use of the information.

Purpose:

The Social Security Administration (SSA) is seeking a COTS product to satisfy the Agency’s Open Source Policy that mandates that third party libraries used in development of agency applications undergo security testing.  SSA is currently using the Sonatype Nexus IQ Server product.  However, this tool does not meet data sorting, data retention and reporting requirements.  The purpose of this RFI is to identify potential vendors capable of meeting SSA’s requirements through a software composition analysis tool that is capable of evaluating applications for insecure third party libraries and licensing agreements.    
Technical Requirements:

•	The solution shall identify and rank security vulnerabilities associated with open source third party libraries 
•	The solution shall provide additional context on vulnerabilities, including whether an exploit exists, potential attack vectors, and time to triage vulnerabilities.  Please specify if this information is native to the proposed solution or if a third party tool is required for this information. 
•	The solution shall provide prioritization rankings for which vulnerabilities to address first.
•	The solution shall provide intuitive dashboard for viewing results
•	The solution shall support the ability to filter security testing results
•	The solution shall support the ability to generate customizable reports
•	The solution shall support the ability to show the next non-vulnerable version of the component and/or the lowest risk version that is available.  
•	The solution shall integrate with Jenkins and Fortify products
•	The solution shall support the ability to generate custom reports about scans and projects and tailor them for different audiences (e.g., Project Managers, First Lines Supervisors, Executives, etc.)
•	The solution shall retain application scan data for one year
•	The solution shall support the ability to generate the following information:
o	which applications are using vulnerable versions of components, 
o	the security scores for each application 
o	the age of each vulnerable component
o	the age and severity of vulnerabilities
o	location of the vulnerability in the code
o	CVEs of the affected application

Support:

•	Product maintenance to include downloadable upgrades, updates, fixes, and patches to the software solution.
•	Product documentation to include guides and manuals for the solution.
•	Telephone, email, and web based support.

Vendor Responses:

Respondents must submit capability statements and information describing the general approach/solution to addressing the listed requirements. General marketing information or reference to vendor web sites will not be considered responsive to this notice.

Interested firms with the capability of providing the requirement shall submit capability statements that demonstrate their expertise in the above-described areas in sufficient detail, including any other specific and relevant information, so the Government can determine the firm’s experience and capability to provide the requirements. Failure to demonstrate the capability of providing the requirement in response to this market survey may affect the Government’s review of the industry’s ability to perform or provide these requirements.
All capability statements shall include the completed response matrix and questions below:
Requirement
Yes
No
Explain how solution meeting requirement
The solution shall identify and rank security vulnerabilities associated with open source third party libraries



The solution shall provide additional context on vulnerabilities, including whether an exploit exists, potential attack vectors, and time to triage vulnerabilities.  Please specify if this information is native to the proposed solution or if a third party tool is required for this information.



The solution shall provide prioritization rankings for which vulnerabilities to address first.



The solution shall provide intuitive dashboard for viewing results



The solution shall support the ability to filter security testing results



The solution shall support the ability to generate customizable reports



The solution shall support the ability to show the next non-vulnerable version of the component



The solution shall integrate with Jenkins and Fortify products



The solution shall support the ability to generate custom reports about scans and projects and tailor them for different audiences (e.g., Project Managers, First Lines Supervisors, Executives, etc.)



The solution shall retain application scan data for one year



The solution shall support the ability to generate the following information:
o	which applications are using vulnerable versions of components, 
o	the security scores for each application 
o	the age of each vulnerable component
o	the age and severity of vulnerabilities
o	location of the vulnerability in the code
o	CVEs of the affected application




Questions:
•	Describe your potential solution.
•	Is your solution a commercial-off-the-shelf product?
•	Is your solution Section 508 compliant?  If so, please include a copy of the Voluntary Product Accessibility Template.
•	Is your solution IPv4 and/or IPv6 compliant?
•	Have you successfully implemented your solution?  If so, please provide details on the number of installations, size of the installations, etc. Indicate if the implementation is currently in production or in a testing environment.
•	Can you provide a demo of your potential solution?
•	Do you have any small business designations? Please list if yes.
•	Do you have this solution on a GSA Schedule or other GWAC?  If so, please provide GSA Schedule or GWAC number as well as GSA/GWAC and/or OEM part numbers.
•	If the solution is delivered in a cloud environment (SaaS or other model) is the solution FedRamp Authorized? If so, how many authorizations currently exist? Does the solution use encryption at rest and in transit? What computing resources, storage resources, and network resources should we be planning for this service/item?
•	Are you able to provide on-site support for installation and operation / training for use of your product?
•	Can your product be installed on-premises? If so, what are the supported environments?

Interested sources that believe they have the ability to provide the solution and perform the services listed herein should submit a detailed statement of their capabilities electronically via email to Samantha.Jonjo@ssa.gov . Please reference 28321319RI0000028in the email subject line. No telephone, mail, or fax responses will be accepted. Responses must be received by 12:00 noon  EST on January 2, 2019.

In addition to capability statements, vendors must include the following:

•	Organization Name, Address, Email Address, Website Address, and Telephone Number.
•	Size and Type of Ownership and Socioeconomic Designation for the organization [i.e. Small Business, Small Disadvantaged Business, 8(a), Women-owned Small Businesses, Veteran-owned Small Businesses, Service Disabled Veteran-owned Small Businesses, Historically Underutilized Business Zone Small Businesses, etc.].
•	Availability of products and services on the GSA schedule if applicable.
•	Business experiences.
•	A completed Voluntary Product Accessibility Template (VPAT) of their solution. A blank VPAT can be obtained at: http://www.itic.org/dotAsset/5644ecd2-5024-417f-bc23-a52650f47ef8.doc

Vendors must identify any proprietary information submitted in response to this Request for Information. The proprietary information must be clearly marked as proprietary. The government will not treat any other information as proprietary.

The Government reserves the right to contact, or not contact, any party responding to this notice in order to obtain further information for market research purposes. The government will not notify respondents of the results of the evaluation of the information received and will not return submittals to the sender. All interested sources must respond to any potential future solicitation announcements separately from responses to this Request for Information. This is not a solicitation announcement for proposals and the government will not award a contract resulting directly from this announcement.

''']

# Normalize the Dummy Text

In [8]:
def transform_text(doc):
    """
    Returns stemmed lowercased alpha-only substrings from a string that are b/w 3 and 17 chars long. 
    It keeps the substring `508`.

    Parameters:
        doc (str): the text of a single FBO document.

    Returns:
        words (str): a string of space-delimited lower-case alpha-only words (except for `508`)
    """

    #manually define stop_words to avoid nltk.download('stopwords')
    stop_words = {'a','about','above','after','again','against','ain','all','am','an','and','any','are','aren',"aren't",
    'as','at','be','because','been','before','being','below','between','both','but','by','can','couldn',"couldn't",'d','did',
    'didn',"didn't",'do','does','doesn',"doesn't",'doing','don',"don't",'down','during','each','few','for','from','further',
    'had','hadn',"hadn't",'has','hasn',"hasn't",'have','haven',"haven't",'having','he','her','here','hers','herself','him',
    'himself','his','how','i','if','in','into','is','isn',"isn't",'it',"it's",'its','itself','just','ll','m','ma','me','mightn',
    "mightn't",'more','most','mustn',"mustn't",'my','myself','needn',"needn't",'no','nor','not','now','o','of','off','on','once',
    'only','or','other','our','ours','ourselves','out','over','own','re','s','same','shan',"shan't",'she',"she's",'should',
    "should've",'shouldn',"shouldn't",'so','some','such','t','than','that',"that'll",'the','their','theirs','them','themselves',
    'then','there','these','they','this','those','through','to','too','until','up','ve','very','was','wasn',"wasn't",'we',
    'were','weren',"weren't",'what','when','where','which','while','who','whom','why','will','with','won',"won't",'wouldn',"wouldn't",
    'y','you',"you'd","you'll","you're","you've",'your','yours','yourself','yourselves'}
    no_nonsense_re = re.compile(r'^[a-zA-Z^508]+$')
    if not isinstance(doc, str):
        logging.warning(f'Found a non-string doc type of {type(doc)}:  {doc}')
        return str(doc).lower()
    doc = doc.lower()
    doc = doc.split()
    words = ''
    for word in doc:
        m = re.match(no_nonsense_re, word)
        if m:
            match = m.group()
            if match in stop_words:
                continue
            else:
                match_len = len(match)
                if match_len <= 17 and match_len >= 3:
                    porter = PorterStemmer()
                    stemmed = porter.stem(match)
                    words += stemmed + ' '
    words = words.strip()

    return words

In [9]:
normalized_text = transform_text(text[0])

In [11]:
pred = pickled_model.predict([normalized_text])

In [12]:
pred

array([0])

In [13]:
dec_func = pickled_model.decision_function([normalized_text])

In [14]:
#This confidence score for a sample is the signed distance of that sample to the class-dividing hyperplane.
dec_func

array([-0.20020642])